# Compare the General Performance of CABG
 - Create a dataframe containing the severity of stenosis using FFR, and if the stenosis subtended territory is ischemic.
 - Add the point-based or voxel-based MBF value to the data-frame (or create a new one?)
 - Violin Plots:
    - Pre/Post MBF against ischemic-grafted, non-ischemic-grafted, non-ischemic nongrafted
    - Pre/Post MBF against severity of stenosis (none, mild, moderate, severe)

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.ndimage import gaussian_filter
from utilities import ReadVTUFile, ThresholdInBetween, ThresholdByUpper, LargestConnectedRegion
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

In [ ]:
path = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/InterPatientStudy"
metadata = glob.glob(f"{path}/*.xlsx")
VTUFiles = sorted(glob.glob(f"{path}/*.vtu"))
TerritoryLabels = sorted(glob.glob(f"{path}/*.dat"))
print(VTUFiles)

PatientMetadata = pd.read_excel(metadata[0], dtype={'column_name': str})
print(PatientMetadata)

In [ ]:
patient_ids = sorted(set(PatientMetadata["patient-id"]))
print(patient_ids)

In [ ]:
def ReadLabels(InputLabels, TerritoryTag):
    MBF_Labels = {}
    for tag in TerritoryTag:
        MBF_Labels[tag] =  []
    MBF_Labels["Nongrafted-Stenosis"] = []
    MBF_Labels["Nongrafted"] = []
    with open(InputLabels, "r") as ifile:
        for i, LINE in enumerate(ifile):
            if i == 0: 
                continue
            line = LINE.strip().split()
            label = line[1]
            id_value = int(line[0])
            found = False

            for key in TerritoryTag:
                if key in label: 
                    MBF_Labels[key].append(id_value)
                    found = True
            
            if not found:
                if "NG" in label:
                    MBF_Labels["Nongrafted-Stenosis"].append(id_value)
                    found = True
                else:
                    MBF_Labels["Nongrafted"].append(id_value)

    MBF_Labels = {k.replace('post_', ''):v for k,v in MBF_Labels.items() if len(v)>0}
    
    return MBF_Labels

def robust_mode(data, bins=50):
    counts, bin_edges = np.histogram(data, bins=bins)
    max_bin_index = np.argmax(counts)
    mode_value = (bin_edges[max_bin_index] + bin_edges[max_bin_index + 1]) / 2
    return mode_value

def Normalize(MBF, method="mode", cutoff = 50):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ArrayName = arrayname_
    
    ScalarArray = vtk_to_numpy(MBF.GetPointData().GetArray(ArrayName))
    #HealthyMyo = LargestConnectedRegion(ThresholdByUpper(MBF, ArrayName, cutoff))
    HealthyMyo = LargestConnectedRegion(ThresholdInBetween(MBF, ArrayName, cutoff, np.percentile(ScalarArray,90)))
    #HealthyMyo = LargestConnectedRegion(ThresholdByUpper(HealthyMyo, ArrayName, cutoff))
    ScalarArray_healthy = vtk_to_numpy(HealthyMyo.GetPointData().GetArray(ArrayName))
    
    if method == "mode": # Histogram Analysis
        # Using Freedman-Diaconis Rule to calculate the number of bins based on IQR
        IQR = np.percentile(ScalarArray_healthy, 75) - np.percentile(ScalarArray_healthy, 25)
        bin_width = 2*IQR/(len(ScalarArray_healthy))**(1/3)
        MBF_range = np.max(ScalarArray_healthy) - np.min(ScalarArray_healthy)
        bins = int(MBF_range/bin_width)
        ref_MBF = robust_mode(ScalarArray_healthy, bins)
    elif method == "median":
        ref_MBF = np.median(ScalarArray_healthy)
    elif method == "mean":
        ref_MBF = np.mean(ScalarArray_healthy)
    elif method == "old":
        ref_MBF = np.percentile(ScalarArray, 75)
    elif method == "zscore":
        q1, q2, q3 = np.percentile(ScalarArray_healthy, [25, 50, 75])
        IndexMBFArray = (ScalarArray - q2)/(q3 - q1)
        IndexMBFArray = gaussian_filter(IndexMBFArray, sigma=1)
        IndexMBF = numpy_to_vtk(IndexMBFArray)
        IndexMBF.SetName("IndexMBF")
        MBF.GetPointData().AddArray(IndexMBF)

        return MBF
    else:
        ref_MBF = np.percentile(ScalarArray_healthy, 75)

    IndexMBFArray = ScalarArray/ref_MBF
    IndexMBFArray = gaussian_filter(IndexMBFArray, sigma=1)
    IndexMBF = numpy_to_vtk(IndexMBFArray)
    IndexMBF.SetName("IndexMBF")
    MBF.GetPointData().AddArray(IndexMBF)

    return MBF


def CollectMBFData(MBF, Labels):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
            arrayname_ = MBF.GetPointData().GetArrayName(i)
            if 'scalars' in arrayname_.lower():
                ScalarArray = arrayname_

    AbsMBFData = {}
    IndexMBFData = {}
    for key in Labels.keys():
        AbsMBFData[key] = np.array([])
        IndexMBFData[key] = np.array([])
        for i in Labels[key]:
            territory_ = ThresholdInBetween(MBF, "TerritoryMaps", i, i)
            MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArray))
            iMBF_ = vtk_to_numpy(territory_.GetPointData().GetArray("IndexMBF"))
            AbsMBFData[key] = np.append(AbsMBFData[key], MBF_)
            IndexMBFData[key] = np.append(IndexMBFData[key], iMBF_)
    

    return AbsMBFData, IndexMBFData

def BoxPlot(DataFrame):
    plt.figure(figsize=(8, 5))
    ax = sns.boxplot(x='Territories', y='MBF', hue='Type', data= DataFrame, showfliers=False, palette={'A':'skyblue', 'B':'salmon'})

    ax.set_ylabel("Index MBF", fontsize = 20)
    ax.tick_params(axis='x', rotation=10, labelsize=15)
    ax.legend(loc = 'upper left')
    plt.tight_layout()
    plt.show()

def ViolinPlot(DataFrame):
    plt.figure(figsize=(10, 6))

    sns.violinplot(x='Territories', y='MBF', hue='Type', data = DataFrame, split=True, inner = 'quartiles', bw_adjust = 10, common_norm=True, linewidth= 2, palette={'A':'skyblue', 'B':'salmon'})

    plt.ylabel("Absolute MBF")
    #plt.xlabel("Group")
    plt.legend(title="Pre/Post CABG")
    #plt.ylim([-0.5, 3])
    plt.show()




In [ ]:
print(PatientMetadata[PatientMetadata['patient-id'] == 'SU03']['stenosis tag'])

In [ ]:
data = []
for patient_id in patient_ids:
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    cutoff = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['cutoff'])[0]
    if cutoff == 0:
        cutoff == 142
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    normalization_method = "zscore"
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
    AbsMBFData_A, IndexMBFData_A = CollectMBFData(MBF_A, MBF_Labels)
    AbsMBFData_B, IndexMBFData_B = CollectMBFData(MBF_B, MBF_Labels)
    for i, t in enumerate(tags):
        data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": ischemia[i], "absMBF_A": AbsMBFData_A[t], "absMBF_B": AbsMBFData_B[t], "IndexMBF_A": IndexMBFData_A[t], "IndexMBF_B": IndexMBFData_B[t]}
        data.append(data_)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": False, "absMBF_A": AbsMBFData_A[t], "absMBF_B": AbsMBFData_B[t], "IndexMBF_A": IndexMBFData_A[t], "IndexMBF_B": IndexMBFData_B[t]}
        data.append(data_)
    t = "Nongrafted"
    data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": False, "absMBF_A": AbsMBFData_A[t], "absMBF_B": AbsMBFData_B[t], "IndexMBF_A": IndexMBFData_A[t], "IndexMBF_B": IndexMBFData_B[t]}
    data.append(data_)

df = pd.DataFrame(data)
df


In [ ]:
print("TerritoryTag", "Median", "IQR", "Mean", "std")
index_mbf_ischemic_A = np.concatenate(df[df['is_ischemic'] == True]['IndexMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_ischemic_A, [25, 50, 75])
average = np.mean(index_mbf_ischemic_A)
std = np.std(index_mbf_ischemic_A)
print("ischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_ischemic_B = np.concatenate(df[df['is_ischemic'] == True]['IndexMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_ischemic_B, [25, 50, 75])
average = np.mean(index_mbf_ischemic_B)
std = np.std(index_mbf_ischemic_B)
print("ischemic_grafted_B",q2, q3-q1, average, std)

index_mbf_nonischemic_A = np.concatenate(df[(df['is_ischemic'] == False) & (df['tag'] != 'Nongrafted') & (df['tag'] != "Nongrafted-Stenosis")]['IndexMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_A, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_A)
std = np.std(index_mbf_nonischemic_A)
print("nonischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_nonischemic_B = np.concatenate(df[(df['is_ischemic'] == False) & (df['tag'] != 'Nongrafted') & (df['tag'] != "Nongrafted-Stenosis")]['IndexMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_B, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_B)
std = np.std(index_mbf_nonischemic_B)
print("nonischemic_grafted_B", q2, q3-q1, average, std)

index_mbf_st_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['IndexMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_A)
std = np.std(index_mbf_st_nongrafted_A)
print("stenosed_nongrafted_A", q2, q3-q1, average, std)

index_mbf_st_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['IndexMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_B)
std = np.std(index_mbf_st_nongrafted_B)
print("stenosed_nongrafted_B", q2, q3-q1, average, std)


index_mbf_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted']['IndexMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_A)
std = np.std(index_mbf_nongrafted_A)
print("nongrafted_A", q2, q3-q1, average, std)

index_mbf_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted']['IndexMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_B)
std = np.std(index_mbf_nongrafted_B)
print("nongrafted_B", q2, q3-q1, average, std)


"""
def downsample(array, size=1000):
    if len(array) > size:
        return np.random.choice(array, size, replace=False)
    return array

index_mbf_ischemic_A = downsample(index_mbf_ischemic_A)
index_mbf_ischemic_B = downsample(index_mbf_ischemic_B)
index_mbf_nonischemic_A = downsample(index_mbf_nonischemic_A)
index_mbf_nonischemic_B = downsample(index_mbf_nonischemic_B)
index_mbf_nongrafted_A = downsample(index_mbf_nongrafted_A)
index_mbf_nongrafted_B = downsample(index_mbf_nongrafted_B)
"""

In [ ]:
# Create a DataFrame with each part
df_A = pd.DataFrame({
    'MBF': np.concatenate([index_mbf_ischemic_A, index_mbf_nonischemic_A, index_mbf_st_nongrafted_A, index_mbf_nongrafted_A]),
    'Territory': ['Ischemic Grafted'] * len(index_mbf_ischemic_A) + 
             ['Non-ischemic Grafted'] * len(index_mbf_nonischemic_A) + 
             ['Nongrafted Stenosis'] * len(index_mbf_st_nongrafted_A) +
             ['Nongrafted'] * len(index_mbf_nongrafted_A),
    'Type': ['A'] * (len(index_mbf_ischemic_A) + len(index_mbf_nonischemic_A) + len(index_mbf_st_nongrafted_A) + len(index_mbf_nongrafted_A))
})

df_B = pd.DataFrame({
    'MBF': np.concatenate([index_mbf_ischemic_B, index_mbf_nonischemic_B, index_mbf_st_nongrafted_B, index_mbf_nongrafted_B]),
    'Territory': ['Ischemic Grafted'] * len(index_mbf_ischemic_B) + 
             ['Non-ischemic Grafted'] * len(index_mbf_nonischemic_B) + 
             ['Nongrafted Stenosis'] * len(index_mbf_st_nongrafted_B) +
             ['Nongrafted'] * len(index_mbf_nongrafted_B),
    'Type': ['B'] * (len(index_mbf_ischemic_B) + len(index_mbf_nonischemic_B) + len(index_mbf_st_nongrafted_B) + len(index_mbf_nongrafted_B))
})

# Combine into a single DataFrame
df_long = pd.concat([df_A, df_B], ignore_index=True)

In [ ]:
plt.figure(figsize=(10, 6))

sns.violinplot(x='Territory', y='MBF', hue='Type', data = df_long, split=True, inner = 'quartiles', bw_adjust = 10, common_norm=True, linewidth= 2, palette={'A':'skyblue', 'B':'salmon'})

plt.ylabel("Index MBF")
#plt.xlabel("Group")
plt.legend(title="Pre/Post CABG")
plt.ylim([-3, 4.5])
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
ax = sns.boxplot(x='Territory', y='MBF', hue='Type', data= df_long, showfliers=False, palette={'A':'skyblue', 'B':'salmon'})

ax.set_ylabel("Index MBF", fontsize = 20)
ax.tick_params(axis='x', rotation=0, labelsize=10)
ax.legend(loc = 'upper left')
plt.tight_layout()
plt.show()

In [ ]:
for patient_id in patient_ids:
    print(patient_id)
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    cutoff = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['cutoff'])[0]
    if cutoff == 0:
        cutoff = 142
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    normalization_method = "zscore"
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
    AbsMBFData_A, IndexMBFData_A = CollectMBFData(MBF_A, MBF_Labels)
    AbsMBFData_B, IndexMBFData_B = CollectMBFData(MBF_B, MBF_Labels)
    data = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    for i, t in enumerate(tags):
        array_A = IndexMBFData_A[t]
        array_B = IndexMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        array_A = IndexMBFData_A[t]
        array_B = IndexMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted"
    array_A = IndexMBFData_A[t]
    array_B = IndexMBFData_B[t]
    data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    data = pd.concat([data, data_], ignore_index=True)
    #BoxPlot(data)
    ViolinPlot(data)


# Mapping Pre-CABG on Post-CABG based on the Distribution of Nongrafted regions

In [ ]:
def MapNongrafted(meanA, meanB, stdA, stdB, DataA):
    return meanB + (stdB/stdA)*(DataA - meanA)



for patient_id in patient_ids:
    print(patient_id)
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    cutoff = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['cutoff'])[0]
    if cutoff == 0:
        cutoff = 142
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    normalization_method = "zscore"
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
    AbsMBFData_A, IndexMBFData_A = CollectMBFData(MBF_A, MBF_Labels)
    AbsMBFData_B, IndexMBFData_B = CollectMBFData(MBF_B, MBF_Labels)
    data = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    dataMapped = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    t = "Nongrafted"
    array_A = AbsMBFData_A[t]
    mean_A = np.mean(array_A)
    std_A = np.std(array_A)
    array_B = AbsMBFData_B[t]
    mean_B = np.mean(array_B)
    std_B = np.std(array_B)
    #data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    #data = pd.concat([data, data_], ignore_index=True)
    # Mapping
    #data_mapped_ = pd.DataFrame({"MBF": np.concatenate([MapNongrafted(mean_A, mean_B, std_A, std_B, array_A), array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    #dataMapped = pd.concat([dataMapped, data_mapped_], ignore_index=True)
    for i, t in enumerate(tags):
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
        # Mapping
        data_mapped_ = pd.DataFrame({"MBF": np.concatenate([MapNongrafted(mean_A, mean_B, std_A, std_B, array_A), array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        dataMapped = pd.concat([dataMapped, data_mapped_], ignore_index=True)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
        # Mapping
        data_mapped_ = pd.DataFrame({"MBF": np.concatenate([MapNongrafted(mean_A, mean_B, std_A, std_B, array_A), array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        dataMapped = pd.concat([dataMapped, data_mapped_], ignore_index=True)
    t = "Nongrafted"
    array_A = AbsMBFData_A[t]
    array_B = AbsMBFData_B[t]
    data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    data = pd.concat([data, data_], ignore_index=True)
    # Mapping
    data_mapped_ = pd.DataFrame({"MBF": np.concatenate([MapNongrafted(mean_A, mean_B, std_A, std_B, array_A), array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    dataMapped = pd.concat([dataMapped, data_mapped_], ignore_index=True)
    
    
    #BoxPlot(data)
    ViolinPlot(data)
    print("--- Mapping Results:")
    ViolinPlot(dataMapped)


# Aggregating The Results

In [153]:
dataMapped = []
for patient_id in patient_ids:
    #print(patient_id)
    if patient_id == "SU25":
        continue
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    cutoff = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['cutoff'])[0]
    if cutoff == 0:
        cutoff = 142
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    normalization_method = "zscore"
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = Normalize(ReadVTUFile(file), normalization_method, cutoff/2)
    AbsMBFData_A, IndexMBFData_A = CollectMBFData(MBF_A, MBF_Labels)
    AbsMBFData_B, IndexMBFData_B = CollectMBFData(MBF_B, MBF_Labels)
    
    t = "Nongrafted"
    array_A = AbsMBFData_A[t]
    mean_A = np.mean(array_A)
    std_A = np.std(array_A)
    array_B = AbsMBFData_B[t]
    mean_B = np.mean(array_B)
    std_B = np.std(array_B)
    for i, t in enumerate(tags):
        data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": ischemia[i], "absMBF_A": MapNongrafted(mean_A, mean_B, std_A, std_B, AbsMBFData_A[t]), "absMBF_B": AbsMBFData_B[t]}
        dataMapped.append(data_)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": ischemia[i], "absMBF_A": MapNongrafted(mean_A, mean_B, std_A, std_B, AbsMBFData_A[t]), "absMBF_B": AbsMBFData_B[t]}
        dataMapped.append(data_)
    t = "Nongrafted"
    data_ = {"patient_id": patient_id, "tag": t, "is_ischemic": ischemia[i], "absMBF_A": MapNongrafted(mean_A, mean_B, std_A, std_B, AbsMBFData_A[t]), "absMBF_B": AbsMBFData_B[t]}
    dataMapped.append(data_)

df = pd.DataFrame(dataMapped)
df


,patient_id,tag,is_ischemic,absMBF_A,absMBF_B
0,SU03,Diag,True,"[-1.58393972761084, 0.18660967361753933, -1.58...","[162.0, 155.0, 168.0, 162.0, 162.0, 180.0, 164..."
1,SU03,LAD,False,"[140.06001237066005, 140.06001237066005, 140.0...","[83.0, 122.0, 188.0, 212.0, 153.0, 150.0, 134...."
2,SU03,RCA,True,"[67.46748692029621, 67.46748692029621, 76.3202...","[229.0, 224.0, 226.0, 222.0, 102.0, 108.0, 103..."
3,SU03,Nongrafted-Stenosis,True,"[28.515400093271722, 40.90924590187042, 32.056...","[74.0, 74.0, 74.0, 73.0, 70.0, 72.0, 73.0, 75...."
4,SU03,Nongrafted,True,"[127.66616656206133, 125.89561716083294, 118.8...","[291.0, 177.0, 126.0, 107.0, 95.0, 100.0, 161...."
...,...,...,...,...,...
56,VA02,Nongrafted,True,"[107.58676682448422, 107.58676682448422, 107.5...","[10.0, 50.0, 12.0, 74.0, 94.0, 82.0, 92.0, 76...."
57,VA04,LAD,False,"[21.589923566519452, 21.589923566519452, 21.58...","[24.0, 3.0, 19.0, 8.0, 99.0, 98.0, 94.0, 99.0,..."
58,VA04,LCx,True,"[25.09589236103681, 24.39469860213334, 21.5899...","[90.0, 81.0, 44.0, 4.0, 83.0, 93.0, 82.0, 88.0..."
59,VA04,RCA,False,"[58.75319278840339, 58.75319278840339, 58.7531...","[76.0, 29.0, 79.0, 27.0, 77.0, 73.0, 75.0, 71...."


In [ ]:
# Generic

print("TerritoryTag", "Median", "IQR", "Mean", "std")
index_mbf_ischemic_A = np.concatenate(df[df['is_ischemic'] == True]['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_ischemic_A, [25, 50, 75])
average = np.mean(index_mbf_ischemic_A)
std = np.std(index_mbf_ischemic_A)
print("ischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_ischemic_B = np.concatenate(df[df['is_ischemic'] == True]['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_ischemic_B, [25, 50, 75])
average = np.mean(index_mbf_ischemic_B)
std = np.std(index_mbf_ischemic_B)
print("ischemic_grafted_B",q2, q3-q1, average, std)

index_mbf_nonischemic_A = np.concatenate(df[(df['is_ischemic'] == False) & (df['tag'] != 'Nongrafted') & (df['tag'] != "Nongrafted-Stenosis")]['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_A, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_A)
std = np.std(index_mbf_nonischemic_A)
print("nonischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_nonischemic_B = np.concatenate(df[(df['is_ischemic'] == False) & (df['tag'] != 'Nongrafted') & (df['tag'] != "Nongrafted-Stenosis")]['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_B, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_B)
std = np.std(index_mbf_nonischemic_B)
print("nonischemic_grafted_B", q2, q3-q1, average, std)

index_mbf_st_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_A)
std = np.std(index_mbf_st_nongrafted_A)
print("stenosed_nongrafted_A", q2, q3-q1, average, std)

index_mbf_st_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_B)
std = np.std(index_mbf_st_nongrafted_B)
print("stenosed_nongrafted_B", q2, q3-q1, average, std)


index_mbf_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted']['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_A)
std = np.std(index_mbf_nongrafted_A)
print("nongrafted_A", q2, q3-q1, average, std)

index_mbf_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted']['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_B)
std = np.std(index_mbf_nongrafted_B)
print("nongrafted_B", q2, q3-q1, average, std)



In [155]:
# Seperates those with severe ischemia

ischemic_A = []
ischemic_B = []
non_low_median_A = []
non_low_median_B = []

for _, row in df[df['is_ischemic'] == True].iterrows():
    id = row['patient_id']
    tag = row['tag']
    region_A = row['absMBF_A']
    region_B = row['absMBF_B']
    if np.median(region_A) < 100:
        print(id, '-', tag)
        ischemic_A.append(region_A)
        ischemic_B.append(region_B)
    else:
        non_low_median_A.append(region_A)
        non_low_median_B.append(region_B)

print("TerritoryTag", "Median", "IQR", "Mean", "std")
index_mbf_ischemic_A = np.concatenate(ischemic_A)
q1, q2, q3 = np.percentile(index_mbf_ischemic_A, [25, 50, 75])
average = np.mean(index_mbf_ischemic_A)
std = np.std(index_mbf_ischemic_A)
print("ischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_ischemic_B = np.concatenate(ischemic_B)
q1, q2, q3 = np.percentile(index_mbf_ischemic_B, [25, 50, 75])
average = np.mean(index_mbf_ischemic_B)
std = np.std(index_mbf_ischemic_B)
print("ischemic_grafted_B",q2, q3-q1, average, std)


nonischemic_mask = (df['is_ischemic'] == False) & (df['tag'] != 'Nongrafted') & (df['tag'] != "Nongrafted-Stenosis")

nonischemic_A = list(df[nonischemic_mask]['absMBF_A'].values)
nonischemic_B = list(df[nonischemic_mask]['absMBF_B'].values)

nonischemic_A.extend(non_low_median_A)
nonischemic_B.extend(non_low_median_B)

index_mbf_nonischemic_A = np.concatenate(nonischemic_A)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_A, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_A)
std = np.std(index_mbf_nonischemic_A)
print("nonischemic_grafted_A", q2, q3-q1, average, std)

index_mbf_nonischemic_B = np.concatenate(nonischemic_B)
q1, q2, q3 = np.percentile(index_mbf_nonischemic_B, [25, 50, 75])
average = np.mean(index_mbf_nonischemic_B)
std = np.std(index_mbf_nonischemic_B)
print("nonischemic_grafted_B", q2, q3-q1, average, std)

index_mbf_st_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_A)
std = np.std(index_mbf_st_nongrafted_A)
print("stenosed_nongrafted_A", q2, q3-q1, average, std)

index_mbf_st_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted-Stenosis']['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_st_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_st_nongrafted_B)
std = np.std(index_mbf_st_nongrafted_B)
print("stenosed_nongrafted_B", q2, q3-q1, average, std)


index_mbf_nongrafted_A = np.concatenate(df[df['tag'] == 'Nongrafted']['absMBF_A'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_A, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_A)
std = np.std(index_mbf_nongrafted_A)
print("nongrafted_A", q2, q3-q1, average, std)

index_mbf_nongrafted_B = np.concatenate(df[df['tag'] == 'Nongrafted']['absMBF_B'].values)
q1, q2, q3 = np.percentile(index_mbf_nongrafted_B, [25, 50, 75])
average = np.mean(index_mbf_nongrafted_B)
std = np.std(index_mbf_nongrafted_B)
print("nongrafted_B", q2, q3-q1, average, std)



SU03 - Diag
SU03 - RCA
SU11 - LCx
SU13 - LAD
SU14 - RCA
SU26 - LAD
SU27 - RCA
VA04 - LCx
TerritoryTag Median IQR Mean std
ischemic_grafted_A 72.07587420756933 66.21657267779221 78.62050044311323 55.20280989248644
ischemic_grafted_B 117.0 60.0 125.32265546755472 53.11462775799547
nonischemic_grafted_A 121.86063108971572 68.4600488907769 130.01599267267486 58.53521023190782
nonischemic_grafted_B 130.0 63.0 137.68736034169785 58.512117089610356
stenosed_nongrafted_A 126.47272764953624 65.64876920754517 134.74438641308078 61.7578101515981
stenosed_nongrafted_B 138.0 61.0 147.6646513167111 61.29584556328778
nongrafted_A 131.65768998822287 67.79916265521467 142.4891523087515 58.87283612598003
nongrafted_B 132.0 59.0 141.3896632350848 59.01652582698411


In [ ]:
# Create a DataFrame with each part
df_A = pd.DataFrame({
    'MBF': np.concatenate([index_mbf_ischemic_A, index_mbf_nonischemic_A, index_mbf_st_nongrafted_A, index_mbf_nongrafted_A]),
    'Territory': ['Severe Ischemic Grafted'] * len(index_mbf_ischemic_A) + 
             ['Mild/Non-ischemic Grafted'] * len(index_mbf_nonischemic_A) + 
             ['Nongrafted Stenosis'] * len(index_mbf_st_nongrafted_A) +
             ['Nongrafted'] * len(index_mbf_nongrafted_A),
    'Type': ['A'] * (len(index_mbf_ischemic_A) + len(index_mbf_nonischemic_A) + len(index_mbf_st_nongrafted_A) + len(index_mbf_nongrafted_A))
})

df_B = pd.DataFrame({
    'MBF': np.concatenate([index_mbf_ischemic_B, index_mbf_nonischemic_B, index_mbf_st_nongrafted_B, index_mbf_nongrafted_B]),
    'Territory': ['Severe Ischemic Grafted'] * len(index_mbf_ischemic_B) + 
             ['Mild/Non-ischemic Grafted'] * len(index_mbf_nonischemic_B) + 
             ['Nongrafted Stenosis'] * len(index_mbf_st_nongrafted_B) +
             ['Nongrafted'] * len(index_mbf_nongrafted_B),
    'Type': ['B'] * (len(index_mbf_ischemic_B) + len(index_mbf_nonischemic_B) + len(index_mbf_st_nongrafted_B) + len(index_mbf_nongrafted_B))
})

# Combine into a single DataFrame
df_long = pd.concat([df_A, df_B], ignore_index=True)

In [ ]:
plt.figure(figsize=(10, 6))

sns.violinplot(x='Territory', y='MBF', hue='Type', data = df_long, split=True, inner = 'quartiles', bw_adjust = 10, common_norm=True, linewidth= 2, palette={'A':'skyblue', 'B':'salmon'})

plt.ylabel("Absolute MBF")
#plt.xlabel("Group")
plt.legend(title="Pre/Post CABG")
plt.ylim([-80, 400])
plt.show()

# Histogram Matching

In [ ]:
from skimage.exposure import match_histograms

def HistogramMatching(MBF_A, MBF_B, Labels):
    for i in range(MBF_A.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF_A.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ScalarArrayName_A = arrayname_

    for i in range(MBF_B.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF_B.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ScalarArrayName_B = arrayname_

    Nongrafted_A = np.array([])
    for i in Labels["Nongrafted"]:
        territory_ = ThresholdInBetween(MBF_A, "TerritoryMaps", i, i)
        MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArrayName_A))
        Nongrafted_A = np.append(Nongrafted_A, MBF_)
        
    Nongrafted_B = np.array([])
    for i in Labels["Nongrafted"]:
        territory_ = ThresholdInBetween(MBF_B, "TerritoryMaps", i, i)
        MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArrayName_B))
        Nongrafted_B = np.append(Nongrafted_B, MBF_)

    ScalarArray_A = vtk_to_numpy(MBF_A.GetPointData().GetArray(ScalarArrayName_A))
    #ScalarArray_B = vtk_to_numpy(MBF_B.GetPointData().GetArray(ScalarArrayName_B))

    #matchedNongrafted_A = match_histograms(Nongrafted_A, Nongrafted_B)
    matchedScalarArray_A = match_histograms(ScalarArray_A, Nongrafted_B)
    NewArray_A = numpy_to_vtk(matchedScalarArray_A)
    NewArray_A.SetName("MatchedScalars_A")
    
    MBF_A.GetPointData().RemoveArray(ScalarArrayName_A)
    MBF_A.GetPointData().AddArray(NewArray_A)
    
    
    return MBF_A

def CollectMBFData2(MBF, Labels):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
            arrayname_ = MBF.GetPointData().GetArrayName(i)
            if 'scalars' in arrayname_.lower():
                ScalarArray = arrayname_

    AbsMBFData = {}
    for key in Labels.keys():
        AbsMBFData[key] = np.array([])
        for i in Labels[key]:
            territory_ = ThresholdInBetween(MBF, "TerritoryMaps", i, i)
            MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArray))
            AbsMBFData[key] = np.append(AbsMBFData[key], MBF_)

    return AbsMBFData

for patient_id in patient_ids:
    print(patient_id)
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = ReadVTUFile(file)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = ReadVTUFile(file)
    MBF_A = HistogramMatching(MBF_A, MBF_B, MBF_Labels)
    AbsMBFData_A = CollectMBFData2(MBF_A, MBF_Labels)
    AbsMBFData_B = CollectMBFData2(MBF_B, MBF_Labels)
    data = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    dataMapped = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    for i, t in enumerate(tags):
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted"
    array_A = AbsMBFData_A[t]
    array_B = AbsMBFData_B[t]
    data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    data = pd.concat([data, data_], ignore_index=True)
    
    ViolinPlot(data)


# Another Mapping Method

In [ ]:
from skimage.exposure import match_histograms

def HistogramMapping(MBF_A, MBF_B, Labels):
    for i in range(MBF_A.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF_A.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ScalarArrayName_A = arrayname_

    for i in range(MBF_B.GetPointData().GetNumberOfArrays()):
        arrayname_ = MBF_B.GetPointData().GetArrayName(i)
        if 'scalars' in arrayname_.lower():
            ScalarArrayName_B = arrayname_

    Nongrafted_A = np.array([])
    for i in Labels["Nongrafted"]:
        territory_ = ThresholdInBetween(MBF_A, "TerritoryMaps", i, i)
        MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArrayName_A))
        Nongrafted_A = np.append(Nongrafted_A, MBF_)
        
    Nongrafted_B = np.array([])
    for i in Labels["Nongrafted"]:
        territory_ = ThresholdInBetween(MBF_B, "TerritoryMaps", i, i)
        MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArrayName_B))
        Nongrafted_B = np.append(Nongrafted_B, MBF_)

    ScalarArray_A = vtk_to_numpy(MBF_A.GetPointData().GetArray(ScalarArrayName_A))
    #ScalarArray_B = vtk_to_numpy(MBF_B.GetPointData().GetArray(ScalarArrayName_B))

    #matchedNongrafted_A = match_histograms(Nongrafted_A, Nongrafted_B)
    src_sorted = np.sort(ScalarArray_A)
    ref_sorted = np.sort(Nongrafted_B)

    src_cdf = np.linspace(0, 1, len(src_sorted))
    ref_cdf = np.linspace(0, 1, len(ref_sorted))

    # Build mapping function
    interp_ref = np.interp(src_cdf, ref_cdf, ref_sorted)

    # Map each scalar in source to its mapped value using inverse CDF lookup
    indices = np.searchsorted(src_sorted, ScalarArray_A)
    indices = np.clip(indices, 0, len(interp_ref) - 1)

    matchedScalarArray_A = interp_ref[indices]
    NewArray_A = numpy_to_vtk(matchedScalarArray_A)
    NewArray_A.SetName("MatchedScalars_A")
    
    MBF_A.GetPointData().RemoveArray(ScalarArrayName_A)
    MBF_A.GetPointData().AddArray(NewArray_A)
    
    
    return MBF_A

def CollectMBFData2(MBF, Labels):
    for i in range(MBF.GetPointData().GetNumberOfArrays()):
            arrayname_ = MBF.GetPointData().GetArrayName(i)
            if 'scalars' in arrayname_.lower():
                ScalarArray = arrayname_

    AbsMBFData = {}
    for key in Labels.keys():
        AbsMBFData[key] = np.array([])
        for i in Labels[key]:
            territory_ = ThresholdInBetween(MBF, "TerritoryMaps", i, i)
            MBF_ = vtk_to_numpy(territory_.GetPointData().GetArray(ScalarArray))
            AbsMBFData[key] = np.append(AbsMBFData[key], MBF_)

    return AbsMBFData

for patient_id in patient_ids:
    print(patient_id)
    tags = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['stenosis tag'])
    ischemia = list(PatientMetadata[PatientMetadata['patient-id'] == patient_id]['is-ischemic'])
    TerritoryTags = [f'post_{tag}' for tag in tags]
    for file in TerritoryLabels:
        if patient_id in os.path.basename(os.path.splitext(file)[0]):
            LabelFile = file
    MBF_Labels = ReadLabels(LabelFile, TerritoryTags)
    for file in VTUFiles:
        if f"{patient_id}A" in os.path.basename(os.path.splitext(file)[0]):
            MBF_A = ReadVTUFile(file)
        elif f"{patient_id}B" in os.path.basename(os.path.splitext(file)[0]):
            MBF_B = ReadVTUFile(file)
    MBF_A = HistogramMatching(MBF_A, MBF_B, MBF_Labels)
    AbsMBFData_A = CollectMBFData2(MBF_A, MBF_Labels)
    AbsMBFData_B = CollectMBFData2(MBF_B, MBF_Labels)
    data = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    dataMapped = pd.DataFrame({"MBF": [], "Territories": [], "Type": []})
    for i, t in enumerate(tags):
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted-Stenosis"
    if t in MBF_Labels.keys():
        array_A = AbsMBFData_A[t]
        array_B = AbsMBFData_B[t]
        data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
        data = pd.concat([data, data_], ignore_index=True)
    t = "Nongrafted"
    array_A = AbsMBFData_A[t]
    array_B = AbsMBFData_B[t]
    data_ = pd.DataFrame({"MBF": np.concatenate([array_A, array_B]), "Territories": [t]*len(array_A) + [t]*len(array_B), "Type": ['A']*len(array_A) + ['B']*len(array_B)})
    data = pd.concat([data, data_], ignore_index=True)
    
    ViolinPlot(data)
